In [190]:
# load dataset
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn.datasets import fetch_california_housing
data_set = fetch_california_housing(as_frame=True)
data=data_set.data
data['target']=data_set.target
data

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [191]:
#data cleaning
data.isnull().sum()
data.duplicated().sum()
## no null and duplicated values

z_score=np.abs(zscore(data))
outlier=data[(z_score>3).any(axis=1)]
data=data.drop(outlier.index,axis=0)
data

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [192]:
# scaling
from sklearn.preprocessing import StandardScaler

colum=data.columns
scaler=StandardScaler()
data=scaler.fit_transform(data)
data=pd.DataFrame(data,columns=colum)

y=data['target']
data=data.drop('target',axis=1)

In [193]:
# model building with cross validation
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.metrics import mean_squared_error,r2_score

model1=LinearRegression()

score1=cross_val_score(model1,data,y,cv=5)
print("Linear regression".center(50,"_"))
print(score1)
print(score1.mean())
y_pred1=cross_val_predict(model1,data,y,cv=5)
print('\nMSE\n',mean_squared_error(y,y_pred1))
print('R-square',r2_score(y,y_pred1))


model2=Ridge(alpha=0.5)
score2=cross_val_score(model2,data,y,cv=5)
print("\nRidge regression".center(50,"_"))
print(score2)
print(score2.mean())
y_pred2=cross_val_predict(model2,data,y,cv=5)
print('\nMSE\n',mean_squared_error(y,y_pred2))
print('R-square',r2_score(y,y_pred2))


________________Linear regression_________________
[0.54629029 0.5197316  0.59020723 0.56192711 0.66475558]
0.576582361180184

MSE
 0.39539598927636604
R-square 0.6046040107236338
________________
Ridge regression_________________
[0.54632379 0.51972645 0.59020985 0.56189815 0.66475576]
0.5765828007658863

MSE
 0.3953965301972372
R-square 0.6046034698027627


In [194]:
# without cv
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data,y,test_size=0.2,random_state=42)
model1.fit(x_train,y_train)

y_pred=model1.predict(x_test)
print(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred))

0.3856853390796632
0.616227147818564
